In [1]:
import tensorflow as tf
import keras
import glob
import cv2
import numpy as np
from keras import Sequential,layers

In [2]:
print('hello')

hello


In [3]:
def load_all_images_rgb(p,shape):
    files = []
 
    for file_name in glob.iglob(p, recursive=True):
        files.append(file_name)
    images = [cv2.cvtColor(cv2.imread(file), cv2.COLOR_BGR2RGB) for file in files]
    if shape:
        return [cv2.resize(img, shape) for img in images]
    else:
        return images
    

In [4]:
shape = (299, 299)

 

In [6]:
batch_size=5
img_height = 20
img_width = 10
data_dir = 'traffic_light_dataset'
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 2440 files belonging to 4 classes.
Using 1952 files for training.
Found 2440 files belonging to 4 classes.
Using 488 files for validation.


In [7]:
class_names = train_ds.class_names
print(class_names)

['0_green', '1_yellow', '2_red', '3_not']


In [8]:

train_ds = train_ds.cache().shuffle(1000)

In [9]:
normalization_layer = keras.layers.Rescaling(1./255)

In [10]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 0.9948039


In [11]:
num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [12]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 20, 10, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 20, 10, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 10, 5, 16)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 5, 32)         4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 2, 32)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 5, 2, 64)          1

In [14]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/10
391/391 [==============================] - 16s 39ms/step - loss: 0.2293 - accuracy: 0.9273 - val_loss: 0.0356 - val_accuracy: 0.9898
Epoch 2/10
391/391 [==============================] - 17s 44ms/step - loss: 0.0443 - accuracy: 0.9846 - val_loss: 0.0165 - val_accuracy: 0.9959
Epoch 3/10
391/391 [==============================] - 11s 29ms/step - loss: 0.0236 - accuracy: 0.9918 - val_loss: 0.0098 - val_accuracy: 0.9959
Epoch 4/10
391/391 [==============================] - 14s 37ms/step - loss: 0.0170 - accuracy: 0.9939 - val_loss: 0.0066 - val_accuracy: 0.9959
Epoch 5/10
391/391 [==============================] - 11s 29ms/step - loss: 0.0147 - accuracy: 0.9954 - val_loss: 0.0175 - val_accuracy: 0.9959
Epoch 6/10
391/391 [==============================] - 15s 39ms/step - loss: 0.0047 - accuracy: 0.9985 - val_loss: 5.6698e-04 - val_accuracy: 1.0000
Epoch 7/10
391/391 [==============================] - 17s 44ms/step - loss: 0.0028 - accuracy: 0.9990 - val_loss: 7.6045e-05 - val_a

In [15]:
model.save('trained_keras.keras')

In [16]:
import matplotlib.pyplot as plt

In [17]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

In [ ]:


plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')


Text(0.5, 1.0, 'Training and Validation Loss')

In [ ]:
plt.show()